In [1]:
import numpy as np
import pandas as pd
import math 
import keras
from keras.datasets import cifar10
from keras.datasets import mnist
import time as t
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from scipy.linalg import pinv2
from keras.utils import to_categorical

In [2]:
(xtrain, ytrain), (xtest, ytest) = mnist.load_data()
y_train = to_categorical(ytrain)
y_test = to_categorical(ytest)
xtrain = xtrain.reshape(-1, 784)
xtest = xtest.reshape(-1, 784)
b = 0.02

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
def predict(inputs,w):
    inputs = inputs.astype(float)
    output = sig(np.dot(inputs, w) + b) 
    return output

def sig(x):
    act = 1/(1+np.exp(-x))
    return act

def accuracy(originalLabel,predicted):
  cor = 0
  t = predicted.shape[0]
  for i in range(t):
    p = np.argmax(predicted[i])
    atl = np.argmax(originalLabel[i])
    cor += 1 if p == atl else 0
  accuracy = cor/t 
  return accuracy

In [4]:
def elmclassifier(xtrain,ytrain):  
    lMax = 10
    E = ytrain
    l = 0
    defAccuracy = 0.8
    while l < lMax :
        l = l + 1
        hmat = np.zeros(shape = (xtrain.shape[0],l))
        for row in xtrain:
           k=0
           row = np.reshape(row, (1,np.product(row.shape)))
           h = []
           for i in range(l):
               w = np.random.random((xtrain.shape[1],1))
               op = predict(row,w)
               h.append(op)  
           h = np.reshape(h, (-1, l))  #l is 6 #this will generate 2d matrix of h
           hmat[k] = h[0]
           k = k + 1
        beta = np.dot(np.linalg.pinv(hmat), ytrain)
        E = E - np.dot(hmat,beta)
        E = sig(E)
        E[E > 0.5] = 1
        E[E <= 0.5] = 0
        acc = accuracy(ytrain,E)
        error = math.sqrt( np.square(np.subtract(ytrain,E)).mean() )
        if(acc > defAccuracy) :
          print("Training Accuracy: ",acc*100,"%")
          print("RMSE: ",error)
        break   
    return beta

In [5]:
def elmclasstest(xtest, ytest,b):
    la = b.shape[0]
    hmat = np.zeros(shape = (10000,la))
    for row in xtest:
       k=0
       row = np.reshape(row, (1,np.product(row.shape)))
       h = []
       for i in range(la):
           w = np.random.random((784,1))
           op = predict(row,w)
           h.append(op)
       h = np.reshape(h, (-1, la))  #l is 6 #this will generate 2d matrix of h
       hmat[k] = h[0]
       k = k + 1  
    o = sig(np.dot(hmat , b))
    o[o > 0.5] = 1
    o[o <= 0.5] = 0
    acc = accuracy(ytest, o)
    error = math.sqrt( np.square(np.subtract(ytest,o)).mean() )
    print("Testing Accuracy",acc*100,"%")
    print("RMSE",error)

In [6]:
classtime = t.time()
beta = elmclassifier(xtrain,ytrain)
classtimeend = t.time()
print("Classification Training time :",abs(classtime-classtimeend))

Training Accuracy:  100.0 %
RMSE:  4.492085632902977
Classification Training time : 3.418935537338257


In [7]:
classtime1 = t.time()
elmclasstest(xtest, ytest,beta)
classtimeend1 = t.time()
print("Classification Testing time :",abs(classtime1-classtimeend1))

Testing Accuracy 100.0 %
RMSE 5.303555411231224
Classification Testing time : 0.5421204566955566


In [8]:
def elmregression(xtrain,ytrain):  
    ytrain = sig(ytrain)
    lMax = 20
    E = ytrain
    l = 0
    defError = 0.1 
    while l < lMax :
        l = l + 1
        hmat = np.zeros(shape = (xtrain.shape[0],l))
        for row in xtrain:
           k=0
           row = np.reshape(row, (1,np.product(row.shape)))
           h = []
           for i in range(l):
               w = np.random.random((xtrain.shape[1],1))
               op = predict(row,w) 
               h.append(op)
           h = np.reshape(h, (-1, l))  #l is 6 #this will generate 2d matrix of h
           hmat[k] = h[0]
           k = k + 1
        beta = np.dot(np.linalg.pinv(hmat), ytrain)
        E = E - np.dot(hmat,beta)
        error = math.sqrt( np.square(np.subtract(ytrain,E)).mean() )
        if(error < defError) :
            print("Training RMSE: ",error)
            break
    return beta

In [9]:
def elmregtest(xtrain, ytrain,b):
    ytrain = sig(ytrain)
    l = b.shape[0]
    hmat = np.zeros(shape = (xtrain.shape[0],l))
    for row in xtrain:
       k=0
       row = np.reshape(row, (1,np.product(row.shape)))
       h = []
       for i in range(l):
           w = np.random.random((xtrain.shape[1],1))
           op = predict(row,w)
           h.append(op)
       h = np.reshape(h, (-1, l)) 
       hmat[k] = h[0]
       k = k + 1 
    sigm = sig(np.dot(hmat , b))
    rmse =  math.sqrt( np.square(np.subtract(ytrain,sigm)).mean() )
    print("Testing RMSE",rmse)

In [12]:
data = pd.read_csv("/content/Folds5x2_pp.csv") 
data.head()

,AT,V,AP,RH,PE
0,8.34,40.77,1010.84,90.01,480.48
1,23.64,58.49,1011.40,74.20,445.75
2,29.74,56.90,1007.15,41.91,438.76
3,19.07,49.69,1007.22,76.79,453.09
4,11.80,40.66,1017.13,97.20,464.43


In [13]:
xtrain = pd.DataFrame(data, columns = ['AT', 'V', 'AP','RH']).to_numpy()
ytrain = pd.DataFrame(data, columns = ['PE']).to_numpy()
aa, bb = ytrain.shape
trainsamples = (int) (aa/2)
testsamples = aa - trainsamples

In [14]:
starttime = t.time()
regb = elmregression(xtrain[0 : trainsamples,:],ytrain[0 : trainsamples,:])
endtime = t.time()
print("Regression training time :",abs(starttime-endtime))
starttime1 = t.time()
elmregtest(xtrain[trainsamples: aa,:],ytrain[testsamples : aa,:],regb)
endtime1 = t.time()
print("Testing time for Regression :",abs(starttime1 - endtime1))

Training RMSE:  0.014457873299156005
Regression training time : 0.19895291328430176
Testing RMSE 0.4999628601357075
Testing time for Regression : 0.19101309776306152
